In [1]:
from generalization.utils import get_model
import torch
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from detection.metrics import *

/mnt/sdd/nguyen.hung.quang/anaconda3/envs/generative/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda:0')

In [3]:
model = get_model('wideresnet')

36479194


In [4]:
model.load_state_dict(torch.load('log/jtt/model_stage_2.pth'))

<All keys matched successfully>

In [19]:
model = torch.load('log/wideresnet10.pth')

/mnt/sdd/nguyen.hung.quang/anaconda3/envs/generative/lib/python3.10/site-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.parallel.data_parallel.DataParallel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/mnt/sdd/nguyen.hung.quang/anaconda3/envs/generative/lib/python3.10/site-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'wideresnet.WideResNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/mnt/sdd/nguyen.hung.quang/anaconda3/envs/generative/lib/python3.10/site-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.m

In [20]:
sum([p.numel() for p in model.parameters()])

36479194

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((125.3/255, 123.0/255, 113.9/255), (63.0/255, 62.1/255.0, 66.7/255.0)),
])


In [6]:
out_dataset = torchvision.datasets.ImageFolder("data/LSUN", transform=transform)
out_loader = DataLoader(out_dataset, batch_size=64, pin_memory=True)

In [7]:
in_dataset = CIFAR10('../data', train=False, transform=transform)
in_loader = DataLoader(in_dataset, batch_size=64, pin_memory=True)

In [8]:
from detection.odin import get_odin_score

In [9]:
import torch.nn as nn
criterion = nn.CrossEntropyLoss()

In [10]:

model = model.to(device)

In [11]:
in_score = get_odin_score(model, in_loader, criterion, device=device)

100%|██████████| 157/157 [00:59<00:00,  2.66it/s]


In [12]:
out_score = get_odin_score(model, out_loader, criterion, device=device)

100%|██████████| 157/157 [00:54<00:00,  2.90it/s]


In [13]:

tpr95(in_score.cpu().numpy(), out_score.cpu().numpy(), 0.1, 0.12)

0.7139999999999999

In [14]:
auroc(in_score.cpu().numpy(), out_score.cpu().numpy(), 0.1, 0.12)

0.8351332100000017

In [15]:
detection_err(in_score.cpu().numpy(), out_score.cpu().numpy(), 0.1, 0.12)

0.2311

In [8]:
model.eval()
model = model.to(device)
pred, labels = [], []
with torch.no_grad():
    for img, label in in_loader:
        img = img.to(device)
        out = model(img)
        pred.append(out.argmax(dim=1))
        labels.append(label)

In [9]:
pred = torch.cat(pred).cpu()
labels = torch.cat(labels).cpu()

In [10]:
(pred == labels).sum() / len(pred)

tensor(0.8262)